In [22]:
import csv
import numpy as np

In [23]:
word_list = []
vocabulary = set()

with open("Preprocessed.csv", encoding="utf8") as file:
    data = csv.reader(file)
    data_list = list(data)

    for tweet in data_list:

        sentence = tweet[2]
        for word in sentence.split():
            word_list.append(word)
            vocabulary.add(word)

print(len(word_list))
print(len(vocabulary))


29111
5736


In [24]:
bigram_list = []
bigram_count = {}
word_count = {}

for i in range(len(word_list)-1):
    bigram = (word_list[i], word_list[i+1])
    bigram_list.append(bigram)

    if bigram in bigram_count:
        bigram_count[bigram] += 1
    else:
        bigram_count[bigram] = 1

    if word_list[i] in word_count:
        word_count[word_list[i]] += 1
    else:
        word_count[word_list[i]] = 1

print(len(bigram_list))
print(len(bigram_count))
print(len(word_count))

29110
26009
5736


In [25]:
bigram_probability_matrix = np.zeros((len(vocabulary), len(vocabulary)))

vocabulary_list = list(vocabulary)
# print(vocabulary_list)
for row in range(len(vocabulary_list)):
    for column in range(len(vocabulary_list)):

        current_bigram = (vocabulary_list[row], vocabulary_list[column])
        current_bigram_count = 0

        if current_bigram in bigram_count:
            current_bigram_count = bigram_count[current_bigram]

        bigram_probability_matrix[row, column] = (current_bigram_count + 1) / (word_count[vocabulary_list[row]] + len(vocabulary))
        # print(word_count[vocabulary_list[row]])

In [26]:
# with np.printoptions(threshold=np.inf):
# np.savetxt("Matrix.csv", bigram_probability_matrix, delimiter =",", fmt ='% s')
# print(bigram_probability_matrix)

[[0.00017431 0.00017431 0.00017431 ... 0.00017431 0.00017431 0.00017431]
 [0.00017428 0.00017428 0.00017428 ... 0.00017428 0.00017428 0.00017428]
 [0.00017422 0.00017422 0.00017422 ... 0.00017422 0.00017422 0.00017422]
 ...
 [0.00017431 0.00017431 0.00017431 ... 0.00017431 0.00017431 0.00017431]
 [0.00017409 0.00017409 0.00017409 ... 0.00017409 0.00017409 0.00017409]
 [0.00017425 0.00017425 0.00017425 ... 0.00017425 0.00017425 0.00017425]]


In [41]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import math
sentiment_analyzer_object = SentimentIntensityAnalyzer()

# vocab_sentiment = {}
bigram_sentiment = np.zeros((len(vocabulary), len(vocabulary)))

# vocabulary_list = list(vocabulary)
# print(vocabulary_list)

for row in range(len(vocabulary_list)):
    for column in range(len(vocabulary_list)):
        bigram = vocabulary_list[row] + " " + vocabulary_list[column]
        sentiment = sentiment_analyzer_object.polarity_scores(bigram)
        bigram_sentiment[row,column] = (abs(math.log10(bigram_probability_matrix[row, column]))) + (abs(sentiment['compound'])*10)
    
# np.savetxt("Matrix.csv", bigram_sentiment, delimiter =",", fmt ='% s')